# PyCitySchools Challenge

In [10]:
import pandas as pd
from pathlib import Path

schools_csv = "../Resources/schools_complete.csv"
students_csv = "../Resources/students_complete.csv"

schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)

district_df = pd.merge(students_df, schools_df, on="school_name", how="left")
district_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


# District Summary

In [11]:
unique_schools = len(district_df["school_name"].unique())
unique_schools

15

In [12]:
student_count = len(district_df["student_name"])
student_count

39170

In [13]:
total_budget = schools_df["budget"].sum()
total_budget

24649428

In [14]:
avg_math_score = district_df["math_score"].mean()
avg_math_score

78.98537145774827

In [15]:
avg_reading_score = district_df["reading_score"].mean()
avg_reading_score

81.87784018381414

In [16]:
count_passed_math = district_df[(district_df["math_score"] >= 70)].count()["student_name"]
percent_passed_math = count_passed_math / student_count * 100
percent_passed_math

74.9808526933878

In [17]:
count_passed_reading = district_df[(district_df["reading_score"] >= 70)].count()["student_name"]
percent_passed_reading = count_passed_reading / student_count * 100
percent_passed_reading

85.80546336482001

In [18]:
count_passed_all = district_df[(district_df["math_score"] >= 70) & (district_df["reading_score"] >= 70)].count()["student_name"]
overall_passed_percent = count_passed_all / student_count * 100
overall_passed_percent

65.17232575950983

In [36]:
district_summary = [{"Total Schools": unique_schools, "Total Students": student_count, "Total Budget": total_budget, 
                    "Average Math Score": avg_math_score, "Average Reading Score": avg_reading_score, "% Passing Math": percent_passed_math,
                    "% Passing Reading": percent_passed_reading, "% Overall Passing": overall_passed_percent}]
district_summary_df = pd.DataFrame(district_summary)

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}%".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


# School Summary